In [1]:
# pip install transformers
# pip install seqeval
# pip install pandas
# pip install numpy
# pip install matplotlib
# pip install seaborn
# !pip install scikit-learn
# !pip install torch torchvision
# !pip install nltk
# !pip install  spacy
# !pip install scipy
# !pip install yellowbrick
# !pip install tqdm
# !pip install openpyxl
# !pip install nlpaug

In [2]:
import pandas as pd
import nltk

In [3]:
path="final data.xlsx"
data=pd.read_excel(path)
data.head()

,sentence,intent
0,Please add a 'Book' node.,add node
1,Make a new node with the label 'Movie'.,add node
2,I would like to add a node labeled 'Game'.,add node
3,Can you create a node with the label 'Animal'?,add node
4,I need a new node with the label 'Plant'.,add node


### DATA AUGMENTATION

BELOW CODE WAS RUN ON GOOGLE COLAB AND COPIED TO HERE

In [4]:
# import nlpaug.augmenter.word as naw
# aug=naw.WordEmbsAug(model_type='word2vec',model_path="/content/supporting models/word2vec/GoogleNews-vectors-negative300.bin",action='substitute')
# count=0
# for sent in data['sentence']:
#   print(sent)
#   print(aug.augment(sent))
#   count+=1
#   if count==3:
#     break

# output:
'''Please add a 'Book' node.
["Persons_wishing relegate a ' Book ' 4Gbit_s_Fibre_Channel."]
Make a new node with the label 'Movie'.
["Reinforce a new node withtheir from Crosstown_Rebels ' Movie '."]
I would like to add a node labeled 'Game'.
["'ve dared Hmm. to add a node labeled ' Playoff_Photos '."]'''

'Please add a \'Book\' node.\n["Persons_wishing relegate a \' Book \' 4Gbit_s_Fibre_Channel."]\nMake a new node with the label \'Movie\'.\n["Reinforce a new node withtheir from Crosstown_Rebels \' Movie \'."]\nI would like to add a node labeled \'Game\'.\n["\'ve dared Hmm. to add a node labeled \' Playoff_Photos \'."]'

In [5]:
# bertAug=naw.ContextualWordEmbsAug(model_path='bert-base-uncased',action='substitute')
# count=0
# for sent in data['sentence']:
#   print(sent)
#   print(bertAug.augment(sent)[0])
#   count+=1
#   if count==3:
#     break
  
#output
'''Please add a 'Book' node.
then contact other'book'node.
Make a new node with the label 'Movie'.
has one new album with the name'movie '.
I would like to add a node labeled 'Game'.
some would like too add one section labeled'game '.'''

"Please add a 'Book' node.\nthen contact other'book'node.\nMake a new node with the label 'Movie'.\nhas one new album with the name'movie '.\nI would like to add a node labeled 'Game'.\nsome would like too add one section labeled'game '."

augmentation is not working i am giving up.

In [56]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

### DATA PRE-PROCESSING

As BERT is trained on a corpus of full sentences of natural language and the embedding process involves contextual embedding, performing stop-word removal might interfere with the accurate functioning of BERT. So we do not remove stop words from our input. This is also why we will not be performing stemming or lemmatization.

In [24]:
import nltk
import transformers

removing all punctuation marks except '' and "" as the specifics of graph manipulation are expected to be input in quotes.

In [17]:
import string
def removePunctuation(sentence):
    cleaned="".join([char for char in sentence if char not in string.punctuation or char=="'" or char=='"'])    
    return cleaned

**tokenization using bert tokenizer** <br>
components: <br>
-->tokenization and add special tokens thatr BERT requires and then add padding <br>
--> attention mask:1 for actual token, 0 for pad tokens<br>
-->convert tikens to ids: after you do step 1, you convert each token to unique ID<br>
-->each token is converted into a vector of (seqlen x 768) dimension <br>
--> feed the attention mask and token IDs to the model after converting them to tensors and unsqueezing them at 0th index ??? <br>
--> op= model(tokenIds,attention_mask) <br>
the op is the final embdded vector

hugging face's BERT tokenizer uses pre trained BERT base, a small version of BERT. we use cased BERT because we want our model to be case sensitive, which will be important for paramter extraction in the named entity recognition layer.

**PRE-PROCESSING FOR BERT** <br>
1. [CLS] and [SEP] are special tokens that BERT requires. this is because BERT was roginally trained for Q-A tasks. For classfn tasks, BERT requirws another special token, which is all in-built in the hugging face tokenizer. 
2. All sequences must be of the same length, so we need to do padding. We do the padding on the right side which is preferred for BERT.
3. Finally we pass in attention mask to assign importance to each token. So all non-padding tokens have an attnetion of 1 and oadding tokens have 0.

In [57]:
from transformers import BertTokenizer
def preProcess(sentence):
    tokenizer=BertTokenizer.from_pretrained('bert-base-cased')
    encoding=tokenizer.encode_plus(
        sentence,
        add_special_tokens= True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt' #pt signifies pyTorch, use tf for tensorFlow & keras
    )
    return encoding

In [58]:
ex="this is a sentence please encode it"
processed=preProcess(ex)

MODEL ARCHITECTURE